In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

`gensim` provides a well-documented implementation of `word2vec`.

In [4]:
from gensim import utils
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec

Logistic regression is a sufficient classification method for word-vectors.

In [6]:
utils

<module 'gensim.utils' from '/usr/local/lib/python3.5/dist-packages/gensim/utils.py'>